In [1]:
import os
import sys
sys.path.append(os.path.abspath('/homes/80/anya/Documents/llm_tiny_ideas/super-tiny-lms-outer/super-tiny-lms'))

In [7]:
from datasets import load_dataset

# Load the gsm8k dataset from Hugging Face
gsm8k = load_dataset("openai/gsm8k", "main")
print(f"Original dataset: {gsm8k}")

instruction_following = "Let's think step by step, concisely, and then output the final answer after \"Answer:\"."

def process(example):
    # Append "Think step by step." to the question
    example["question"] = example["question"].strip() + " " + instruction_following
    
    # Split the answer on "####"
    parts = example["answer"].split("####")
    assert len(parts) == 2, f"Expected 2 parts, but got {len(parts)} parts"
    # Add the new fields to the example
    example["reasoning"] = parts[0].strip()
    example["answer"] = parts[1].strip() #.replace(",", "")
    return example

# Apply the processing function to each example in all splits (e.g., "train" and "test")
gsm8k_processed = gsm8k.map(process)
print(f"Processed dataset: {gsm8k_processed}")

# Define a filtering function that keeps examples with digit-only answers
def is_negative(example):
    return not example["answer"].startswith("-")

# Filter out examples where the answer is not purely digit-based
gsm8k_filtered = gsm8k_processed.filter(is_negative)
print(f"Filtered dataset: {gsm8k_filtered}")

save_dir = "/homes/80/anya/Documents/llm_tiny_ideas/coconut-outer/coconut/data/my_data/gsm8k"
gsm8k_processed.save_to_disk(save_dir)


Original dataset: DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})
Processed dataset: DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'reasoning'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer', 'reasoning'],
        num_rows: 1319
    })
})
Filtered dataset: DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'reasoning'],
        num_rows: 7470
    })
    test: Dataset({
        features: ['question', 'answer', 'reasoning'],
        num_rows: 1317
    })
})


Saving the dataset (1/1 shards): 100%|██████████| 1319/1319 [00:00<00:00, 79751.57 examples/s]


In [6]:
start = 1000
for i in range(start, start + 1000):
    answer = gsm8k_processed["train"]["answer"][i]
    if len(answer.split(",")) > 1:
        print(answer)

15,620
9,492
86,400
2,520,000
54,000
885,000
9,240
28,800
59,616
27,000
100,000
3,380
200,000
3,890
1,600
150,000
